# Creating an Agent using Large Language Models (LLMs)

**Goal:** Create a natural language interface for a simple API that reads / write to a redis cache as a proof of concept. 

Examples: 
1. Create a new record for a new user Ryan Skinner, locaed in Denver Colorado
2. Retreieve a record for the user Ryan Skinner

### Basic Configuration of the OpenAI Client

In [6]:
import os
import ssl
import json

from openai import OpenAI
from ddtrace.llmobs import LLMObs

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [7]:
# Load the secrets
with open("secrets.json") as f:
    secrets = json.load(f)
DD_API_KEY = secrets["DD_API_KEY"]
DD_SITE = secrets["DD_SITE"]
OAI_API_KEY = secrets["OAI_API_KEY"]

# Enable the integration
LLMObs.enable(
    integrations_enabled=True, 
    ml_app="skinner-OAi-llm-agent", 
    api_key = DD_API_KEY,
    site = DD_SITE,
    agentless_enabled = True,
    env="test",
    service="llm-agent"
)

# Establish the Client
client = OpenAI(api_key=OAI_API_KEY)


# Establish the LLM Client
llm = ChatOpenAI(api_key=OAI_API_KEY)

## Prompt engineering to classify a users text

In [8]:
system_prompt = "You are responsible for classifying a users request as the following \n\
    1. A request to add a user to the database \n\
    2. Update the user information to the database\n\
    3. Retrieve information about the user from the database\n\n\
The only valid responses are ADD, UPDATE, RETRIEVE, UNKNOWN\n\n\
If the user intends to add a user respond with ADD\n\
If the user intends to update user information in the database respond with UPDATE\n\
If the user inteds to retrieve information from the database respond with RETRIEVE\n\
If unknown or uncertain respond with UNKNOWN"
print(system_prompt)

You are responsible for classifying a users request as the following 
    1. A request to add a user to the database 
    2. Update the user information to the database
    3. Retrieve information about the user from the database

The only valid responses are ADD, UPDATE, RETRIEVE, UNKNOWN

If the user intends to add a user respond with ADD
If the user intends to update user information in the database respond with UPDATE
If the user inteds to retrieve information from the database respond with RETRIEVE
If unknown or uncertain respond with UNKNOWN


In [9]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user","{input}")
])

In [10]:
chain = prompt | llm
chain.invoke("Where does Ryan live?")

AIMessage(content='RETRIEVE', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 136, 'total_tokens': 140}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d2948223-2125-4d1d-a6b4-c280646c6b4b-0', usage_metadata={'input_tokens': 136, 'output_tokens': 4, 'total_tokens': 140})

# Creating a new user in the Database
---
If the initial classification response is `ADD` we'll want to prompt the user to supply information about the item that we'll add to the database. 

The API provided expects the following information (name, description, quantity, department, price, and tax): 
```
class Item(BaseModel):
    name: str
    description: str | None = None
    qty: int
    department: str
    price: float
    tax: float
```

We'll expect the agent to parse out this information and create the JSON body for the request

In [21]:
add_user_prompt_txt = "You are an agent responsible for parsing a users request and creating a valid JSON response from the request.You are expected to extract the following information\n\
    1. name: The name of the item\n\
    2. description: The description of the item\n\
    3. qty: The quantity of the item available\n\
    4. department: the department the item belongs to\n\
    5. price: The price of the item\n\
    6. tax: The tax associated with the item\n\n\
For any field that you cannot confidently extra use 'None'\n\n\
Example Request: 'name - Coffee pot, description: Creates coffee, qty: 20, department: homegoods, price: $20, tax 8.25%\n\
You are expected to create a synatically valid JSON response with the provided information"

print(add_user_prompt_txt)

You are an agent responsible for parsing a users request and creating a valid JSON response from the request.You are expected to extract the following information
    1. name: The name of the item
    2. description: The description of the item
    3. qty: The quantity of the item available
    4. department: the department the item belongs to
    5. price: The price of the item
    6. tax: The tax associated with the item

For any field that you cannot confidently extra use 'None'

Example Request: 'name - Coffee pot, description: Creates coffee, qty: 20, department: homegoods, price: $20, tax 8.25%
You are expected to create a synatically valid JSON response with the provided information


In [22]:
add_user_prompt = ChatPromptTemplate.from_messages([
    ("system",add_user_prompt_txt),
    ("user","{input}")
])

In [23]:
chain = add_user_prompt | llm
chain.invoke("name is Spoon, description: used to eat soup, quantity: 10, department kitchen, price: $5")

AIMessage(content='{\n    "name": "Spoon",\n    "description": "used to eat soup",\n    "qty": 10,\n    "department": "kitchen",\n    "price": "$5",\n    "tax": "None"\n}', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 199, 'total_tokens': 248}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f5b4c43f-209c-46a7-848c-47357f3621ea-0', usage_metadata={'input_tokens': 199, 'output_tokens': 49, 'total_tokens': 248})